In [865]:
# Instala/atualiza bibliotecas que podem não estar instaladas
!pip install bs4
!pip install unidecode
!pip install --upgrade pandas
!pip install --upgrade openpyxl

In [866]:
# Importa as bibliotecas necessárias para excução do código
from bs4 import BeautifulSoup
from unidecode import unidecode
import requests
import pandas as pd
import numpy as np
import re
import os
from functools import reduce
from math import floor

In [867]:
# Declaração das variáveis contendo as URL a ser acessadas para a extração dos dados
URL_MUNICIPIOS = "https://pt.wikipedia.org/wiki/Lista_de_municípios_do_Brasil_por_população_(2020)"
URL_IDHM = "https://www.br.undp.org/content/brazil/pt/home/idh0/rankings/idhm-municipios-2010.html"
URL_CELULARES = "https://www.teleco.com.br/nceluf.asp"
URL_AGENCIAS = "https://drive.google.com/u/0/uc?id=1ETK0_pv1VoBGSBQDCxI-DOLRF8eJ-JQT&export=download"
URL_ATENDIMENTO = "https://drive.google.com/u/0/uc?id=1DIq8-iT_rNiQ4QhIyy4ZreRxAM9temNC&export=download"
URL_ATENDIMENTO_ELETRONICO = "https://drive.google.com/u/0/uc?id=1357U7jrFv20rTPhGDnsbYDvZIgE01CCU&export=download"

# Acesso às páginas URL descritas anteriormente e salvamento de seus conteúdos em novas variáveis
html_municipios = requests.get(URL_MUNICIPIOS).text
html_idhm = requests.get(URL_IDHM).text
html_celulares = requests.get(URL_CELULARES).text

# Criação de objetos soup com os conteúdos HTML, ficando mais fácil para manipular
soup_municipios = BeautifulSoup(html_municipios, "html.parser")
soup_idhm = BeautifulSoup(html_idhm, "html.parser")
soup_celulares = BeautifulSoup(html_celulares, "html.parser")

In [868]:
# Procura, em cada objeto soup, por tabelas (tag <table/>)
# Como elas são identificadas por suas classes, isso se torna um filtro na busca
lista_tabelas_municipios = soup_municipios.find_all("table", {"class": "wikitable"})
lista_tabelas_idhm = soup_idhm.find_all("table", {"class": "tableizer-table"})
lista_tabelas_celulares = soup_celulares.find_all("table")

# Quantidade de tabelas encontradas em cada site
# print(f"Número de tabelas em {URL_MUNICIPIOS}: {len(lista_tabelas_municipios)}")
# print(f"Número de tabelas em {URL_IDHM}: {len(lista_tabelas_idhm)}")
# print(f"Número de tabelas em {URL_CELULARES}: {len(lista_tabelas_celulares)}")
# Nota-se que no url URL_CELULARES foram encontradas 8 tabelas por elas não terem classes para filtro, diferentemente dos outros dois sites que encontraram apenas 1

In [869]:
# Salva a tabela em uma variável, para usá-la fora da lista
tabela_municipios = lista_tabelas_municipios[0]

# Procura todas as linhas
body_municipios = tabela_municipios.find_all("tr")

# Salva o cabeçalho em uma variável
headings_municipios = body_municipios[0]

# Salva o resto das linhas em outra variável
rows_municipios = body_municipios[1:]

# Cria lista para salvar os nomes de colunas de municipios
headings_municipios_final = list()

# Faz um loop pelas tags <th/> de cabeçalho para salvá-los na lista anterior
for item in headings_municipios.find_all("th"):

  # Retira as quebras de linha e salva na lista
  item = (item.text).rstrip("\n")
  headings_municipios_final.append(item)

In [870]:
# Cria lista para salvar as entradas de municípios
rows_municipios_final = list()

# Faz um loop para percorrer cada linha
for row_atual_indice in range(len(rows_municipios)):

  # Cria lista para salvar as células da linha atual (é zerada a cada passo do loop)
  row_municipios_atual = list()

  # Faz novo loop para percorrer cada célula com tag <td/> na linha atual
  for item in rows_municipios[row_atual_indice].find_all("td"):

    # Deleta bandeira, quebras de linhas e vírgulas do texto da célula
    item = re.sub("(\xa0)|(\n)|,", "", item.text)

    # Adiciona o texto final à lista da linha atual
    row_municipios_atual.append(item)
    
  # Após todas as células estarem na linha atual, adiciona ela à lista de todas as linhas da tabela
  rows_municipios_final.append(row_municipios_atual)

In [871]:
# Dicionário contendo as unidades federativas do Brasil e suas siglas
DICT_UF = {
           "Acre": "AC",
           "Alagoas": "AL",
           "Amazonas": "AM",
           "Amapá": "AP",
           "Bahia": "BA",
           "Ceará": "CE",
           "Distrito Federal": "DF",
           "Espírito Santo": "ES",
           "Goiás": "GO",
           "Maranhão": "MA",
           "Mato Grosso": "MT",
           "Mato Grosso do Sul": "MS",
           "Minas Gerais": "MG",
           "Pará": "PA",
           "Paraíba": "PB",
           "Paraná": "PR",
           "Pernambuco": "PE",
           "Piauí": "PI",
           "Rio de Janeiro": "RJ",
           "Rio Grande do Norte": "RN",
           "Rio Grande do Sul": "RS",
           "Rondônia": "RO",
           "Roraima": "RR",
           "Santa Catarina": "SC",
           "São Paulo": "SP",
           "Sergipe": "SE",
           "Tocantins": "TO"
           }

# Função que retorna a sigla da unidade federativa, dado o seu nome
def conversao_uf(uf: str) -> str: 
  return DICT_UF[uf]

In [872]:
# Cria um DataFrame com os dados tratados da tabela de municípios
df_municipios = pd.DataFrame(data=rows_municipios_final, columns=headings_municipios_final)

In [873]:
# Remove as colunas Posição e Código IBGE, pois não são de interesse
df_municipios = df_municipios.drop(columns=["Posição", "Código IBGE"])

# Renomeia a coluna Unidade federativa para UF
df_municipios = df_municipios.rename(columns={"Unidade federativa": "UF"})

# Converte os números em forma de string da coluna População para tipo numérico
df_municipios["População"] = df_municipios["População"].apply(pd.to_numeric)

# Retira os acentos dos nomes dos municípios e os coloca em maiúsculo, para posterior facilitação na junção de DataFrames
df_municipios["Município"] = df_municipios["Município"].apply(unidecode)
df_municipios["Município"] = df_municipios["Município"].str.upper()

# Converte os nomes das UF para siglas, usando a função conversao_uf()
df_municipios["UF"] = df_municipios["UF"].apply(conversao_uf)

df_municipios = df_municipios.reindex(columns=["UF", "Município", "População"])

In [874]:
# Mostra as 5 primeiras entradas do DataFrame
df_municipios.head()

,UF,Município,População
0,SP,SAO PAULO,12325232
1,RJ,RIO DE JANEIRO,6747815
2,DF,BRASILIA,3055149
3,BA,SALVADOR,2886698
4,CE,FORTALEZA,2686612


In [875]:
# Repete o mesmo processo para a tabela do IDHM
tabela_idhm = lista_tabelas_idhm[0]

body_idhm = tabela_idhm.find_all("tr")

headings_idhm = body_idhm[0]

rows_idhm = body_idhm[1:]

headings_idhm_final = list()

for item in headings_idhm.find_all("th"):
  item = re.sub("\n", " ", item.text)
  headings_idhm_final.append(item)

# Adiciona o cabeçalho UF à lista, para se tornar uma coluna posteriormente
headings_idhm_final.append("UF")

In [876]:
# Continua o mesmo processo
rows_idhm_final = list()

# String contendo as UF do Brasil
siglas_uf = "(AC)|(AL)|(AM)|(AP)|(BA)|(CE)|(DF)|(ES)|(GO)|(MA)|(MT)|(MS)|(MG)|(PA)|(PB)|(PR)|(PE)|(PI)|(RJ)|(RN)|(RS)|(RO)|(RR)|(SC)|(SP)|(SE)|(TO)"

for row_atual_indice in range(len(rows_idhm)):

  # Variável que salvará a UF do município atual
  uf_atual = str()
  row_idhm_atual = list()
  for item in rows_idhm[row_atual_indice].find_all("td"):

    # Verifica qual é a UF atual e salva na variável correspondente
    # A verificação é feita desta forma porque esta tabela segue a seguinte formatação:
    # Município (UF)
    for uf in DICT_UF.values():
      if uf in item.text:
        uf_atual = uf

    # Remove as UF presentes nos nomes das cidades nessa tabela
    item = (item.text).rstrip(siglas_uf)

    # Remove os espaços em branco no final das strings
    item = item.rstrip()
    row_idhm_atual.append(item)
  row_idhm_atual.append(uf_atual)
  rows_idhm_final.append(row_idhm_atual)

In [877]:
# Cria um DataFrame com os dados tratados da tabela de IDHM
df_idhm = pd.DataFrame(data=rows_idhm_final, columns=headings_idhm_final)

In [878]:
# Remove essas colunas, pois não são de interesse
df_idhm = df_idhm.drop(columns=["Ranking IDHM 2010", "IDHM Renda 2010", "IDHM Longevidade 2010", "IDHM Educação 2010"])

# Renomeia IDHM 2010 para apenas IDHM
df_idhm = df_idhm.rename(columns={"IDHM 2010": "IDHM"})

# Substitui as vírgulas dos números decimais por pontos
df_idhm["IDHM"] = df_idhm["IDHM"].str.replace(",", ".")

# Converte os números em forma de string da coluna IDHM para tipo numérico
df_idhm["IDHM"] = df_idhm["IDHM"].apply(pd.to_numeric)

# Retira os acentos dos nomes dos municípios e os coloca em maiúsculo, para posterior facilitação na junção de DataFrames
df_idhm["Município"] = df_idhm["Município"].apply(unidecode)
df_idhm["Município"] = df_idhm["Município"].str.upper()

# Reordena as colunas
df_idhm = df_idhm.reindex(columns=["UF", "Município", "IDHM"])

In [879]:
# Mostra as 5 primeiras entradas do DataFrame
df_idhm.head()

,UF,Município,IDHM
0,SP,SAO CAETANO DO SUL,0.862
1,SP,AGUAS DE SAO PEDRO,0.854
2,SC,FLORIANOPOLIS,0.847
3,SC,BALNEARIO CAMBORIU,0.845
4,ES,VITORIA,0.845


In [880]:
# Como falado acima, não há classe nas tabelas em URL_CELULARES, ou seja, não há como filtrar
# Sendo assim, todas as tabelas da página foram para a lista lista_tabelas_celulares
# Então, é feito um loop para encontrar aquela de interesse, quem contém a palavra "Estado" como um cabeçalho
for tabela in lista_tabelas_celulares:
  body_tabela = tabela.find_all("tr")
  headings_tabela = body_tabela[0]
  for item in headings_tabela.find_all("th"):
    item = (item.text).rstrip("\n")
    # Condição para a parada do loop
    if item == "Estado":
      tabela_celulares = tabela
      break

In [881]:
# Assim, o processo é repetido
body_celulares = tabela_celulares.find_all("tr")

# É salvo a segunda linha de cabeçalhos pois a coluna de interesse encontra-se nessa posição
headings_celulares = body_celulares[1]

# As entradas são a partir da terceira linha
rows_celulares = body_celulares[2:]

headings_celulares_final = list()

for item in headings_celulares.find_all("th"):
  # Tratamento para correção de espaços em branco excessivos
  item = re.sub("\n|\r", " ", item.text)
  item = item.replace(" ", "")
  headings_celulares_final.append(item)

# Como alguns dos títulos de colunas estavam na primeira linha de cabeçalhos, enquanto a maioria estava na segunda linha (a escolhida),
# eles serão adicionados agora: Estado e Dez 2020 (Milhares), nessa ordem após SMC

# A tabela não é uniforme devido às colunas SMP e SMC. Como elas não são de interesse, serão removidas do cabeçalho

headings_celulares_final.remove("SMP")
headings_celulares_final.remove("SMC")

# Como Estado ficaria logo após SMP e SMC e eles eram os primeiros da lista, ele entra na posição 0
headings_celulares_final.insert(0, "Estado")

# E Dez 2020 (Milhares) entrar na posição 1, por estar após Estado
headings_celulares_final.insert(1, "Dez 2020 (Milhares)")

In [882]:
# O processo continua
rows_celulares_final = list()

for row_atual_indice in range(len(rows_celulares)):
  row_celulares_atual = list()
  for item in rows_celulares[row_atual_indice].find_all("td"):
    item = (item.text).rstrip("\n")
    row_celulares_atual.append(item)

  # Como dito anteriormente, não é uma tabela uniforme e, pela exclusão das colunas SMP e SMC, é preciso remover esses valores aqui também
  # Já que em todas as outras colunas há apenas uma célula por coluna, cada linha deve ter o tamanho do vetor headings_celulares_final
  # Se o tamanho da linha atual tiver duas posições a mais, ela tem valores para SMP e SMC
  if len(row_celulares_atual) == (len(headings_celulares_final) + 2):
    # Como estão nas posições 0 e 1, são removidas
    row_celulares_atual.pop(1)
    row_celulares_atual.pop(0)

  # Caso seja uma posição maior, significa que tem valor apenas para SMC
  elif len(row_celulares_atual) == (len(headings_celulares_final) + 1):

    # Então, apenas a posição é removida
    row_celulares_atual.pop(0)
  rows_celulares_final.append(row_celulares_atual)

In [883]:
# Cria um DataFrame com os dados tratados da tabela de celulares
df_celulares = pd.DataFrame(data=rows_celulares_final, columns=headings_celulares_final)

In [884]:
# Remove essas colunas, pois não são de interesse
df_celulares = df_celulares.drop(columns=["Dez 2020 (Milhares)", "NºCel.", "PréPagos", "Cresc.Mês"])

# Na tabela, as últimas 4 entradas são referentes ao Brasil todo e regiões menores, portanto são excluídas do dataframe com base no seu índice
df_celulares = df_celulares.drop(range(len(df_celulares.index) - 4, len(df_celulares.index)))

# Renomeia Dens* para apenas Teledensidade
df_celulares = df_celulares.rename(columns={"Dens*": "Teledensidade"})

# Substitui as vírgulas dos números decimais por pontos
df_celulares["Teledensidade"] = df_celulares["Teledensidade"].str.replace(",", ".")

# Converte os números em forma de string da coluna Teledensidade para tipo numérico
df_celulares["Teledensidade"] = df_celulares["Teledensidade"].apply(pd.to_numeric)

# Divide todos os valores de TD por 100 pois antes o número era referente a 100 habitantes
df_celulares["Teledensidade"] = df_celulares["Teledensidade"]/100

# Renomeia Estado para UF
df_celulares = df_celulares.rename(columns={"Estado": "UF"})

# Converte os nomes das UF para siglas, usando convresao_uf() novamente
df_celulares["UF"] = df_celulares["UF"].apply(conversao_uf)

In [885]:
# Mostra as 5 primeiras entradas do DataFrame
df_celulares.head()

,UF,Teledensidade
0,RJ,1.1710
1,ES,1.0332
2,MG,1.0290
3,AM,0.9254
4,RR,1.0120


In [886]:
# Cria DataFrame com o arquivo excel referente às agências
df_agencias = pd.read_excel(URL_AGENCIAS)

In [887]:
# Cria lista para guardar as colunas a serem removidas
columns_agencias_drop = list()

# Loop para percorrer todas as colunas do DataFrame
for column in df_agencias:

  # Remove os espaços em branco
  new_column = column.rstrip()

  # Atualiza os nomes das colunas
  df_agencias = df_agencias.rename(columns={column: new_column})

  # Se a coluna for diferente das de interesse, entra na lista de drop
  if new_column != "MUNICíPIO" and new_column != "UF": 
    columns_agencias_drop.append(new_column)

# Remove as colunas presentes na lista de drop
df_agencias = df_agencias.drop(columns=columns_agencias_drop)

# Renomeia as colunas
df_agencias = df_agencias.rename(columns={"MUNICíPIO": "Município"})

# Reorganiza as colunas
df_agencias = df_agencias.reindex(columns=["UF", "Município"])

# Remove espaços em branco das colunas Município e Agência
df_agencias["Município"] = [" ".join(item) for item in df_agencias["Município"].str.split()]

In [888]:
# Mostra as primeiras 5 entradas do DataFrame
df_agencias.head()

,UF,Município
0,AM,MANAUS
1,PA,BELEM
2,SP,SANTOS
3,RJ,CAMPOS DOS GOYTACAZES
4,BA,SALVADOR


In [889]:
# Cria DataFrame com o arquivo excel referente aos postos de atendimento
df_atendimento = pd.read_excel(URL_ATENDIMENTO)

In [890]:
# Cria lista para guardar as colunas a serem removidas
columns_atendimento_drop = list()

# Loop para percorrer todas as colunas do DataFrame
for column in df_atendimento:

  # Remove os espaços em branco
  new_column = column.rstrip()

  # Atualiza os nomes das colunas
  df_atendimento = df_atendimento.rename(columns={column: new_column})

  # Se a coluna for diferente das de interesse, entra na lista de drop
  if new_column != "MUNICIPIO" and new_column != "UF": 
    columns_atendimento_drop.append(new_column)

# Remove as colunas presentes na lista de drop
df_atendimento = df_atendimento.drop(columns=columns_atendimento_drop)

# Renomeia as colunas
df_atendimento = df_atendimento.rename(columns={"MUNICIPIO": "Município"})

# Reorganiza as colunas
df_atendimento = df_atendimento.reindex(columns=["UF", "Município"])

# Remove espaços em branco das colunas Município
df_atendimento["Município"] = [" ".join(item) for item in df_atendimento["Município"].str.split()]

In [891]:
# Mostra as primeiras 5 entradas do DataFrame
df_atendimento.head()

,UF,Município
0,AC,ASSIS BRASIL
1,AC,FEIJO
2,AC,MANCIO LIMA
3,AC,RIO BRANCO
4,AC,RIO BRANCO


In [892]:
# Cria DataFrame com o arquivo excel referente aos postos de atendimento eletrônico
df_atendimento_eletronico = pd.read_excel(URL_ATENDIMENTO_ELETRONICO)

In [893]:
# Cria lista para guardar as colunas a serem removidas
columns_atendimento_eletronico_drop = list()

# Loop para percorrer todas as colunas do DataFrame
for column in df_atendimento_eletronico:

  # Remove os espaços em branco
  new_column = column.rstrip()

  # Atualiza os nomes das colunas
  df_atendimento_eletronico = df_atendimento_eletronico.rename(columns={column: new_column})

  # Se a coluna for diferente das de interesse, entra na lista de drop
  if new_column != "MUNICIPIO" and new_column != "UF": 
    columns_atendimento_eletronico_drop.append(new_column)

# Remove as colunas presentes na lista de drop
df_atendimento_eletronico = df_atendimento_eletronico.drop(columns=columns_atendimento_eletronico_drop)

# Renomeia as colunas
df_atendimento_eletronico = df_atendimento_eletronico.rename(columns={"MUNICIPIO": "Município"})

# Reorganiza as colunas
df_atendimento_eletronico = df_atendimento_eletronico.reindex(columns=["UF", "Município"])

# Remove espaços em branco das colunas Município
df_atendimento_eletronico["Município"] = [" ".join(item) for item in df_atendimento_eletronico["Município"].str.split()]

In [894]:
# Mostra as primeiras 5 entradas do DataFrame
df_atendimento_eletronico.head()

,UF,Município
0,,MIAMI
1,,NEW YORK
2,AC,ACRELANDIA
3,AC,ASSIS BRASIL
4,AC,BRASILEIA


In [895]:
# Conta quantas vezes cada município aparece na tabela de Agência e coloca em um novo dataframe
# Cada entrada na tabela significa uma nova agência
df_agencias_final = df_agencias.groupby(df_agencias.columns.tolist(), as_index=False).size()

# Renomeia a coluna size para Agências
df_agencias_final = df_agencias_final.rename(columns={"size": "Agências"})

# Converte a quantidade de agências para tipo númerico
df_agencias_final["Agências"] = df_agencias_final["Agências"].apply(pd.to_numeric)

In [896]:
# Mostra as primeiras 5 entradas do DataFrame
df_agencias_final.head()

,UF,Município,Agências
0,AC,ACRELANDIA,1
1,AC,ASSIS BRASIL,1
2,AC,BRASILEIA,4
3,AC,BUJARI,1
4,AC,CRUZEIRO DO SUL,4


In [897]:
# O mesmo processo é aplicado para o DataFrame de Postos de Atendimento
df_atendimento_final = df_atendimento.groupby(df_atendimento.columns.tolist(), as_index=False).size()

# Renomeia a coluna size para PA
df_atendimento_final = df_atendimento_final.rename(columns={"size": "PA"})

# Converte a quantidade de agências para tipo númerico
df_atendimento_final["PA"] = df_atendimento_final["PA"].apply(pd.to_numeric)

In [898]:
# Mostra as primeiras 5 entradas do DataFrame
df_atendimento_final.head()

,UF,Município,PA
0,AC,ACRELANDIA,3
1,AC,ASSIS BRASIL,1
2,AC,BRASILEIA,2
3,AC,CAPIXABA,2
4,AC,CRUZEIRO DO SUL,3


In [899]:
# O mesmo procedimento é feito com o DataFrame de Postos de Atendimento Eletrônico
df_atendimento_eletronico_final = df_atendimento_eletronico.groupby(df_atendimento_eletronico.columns.tolist(), as_index=False).size()

# Renomeia a coluna size para PAE
df_atendimento_eletronico_final = df_atendimento_eletronico_final.rename(columns={"size": "PAE"})

# Converte a quantidade de agências para tipo númerico
df_atendimento_eletronico_final["PAE"] = df_atendimento_eletronico_final["PAE"].apply(pd.to_numeric)

In [900]:
# Mostra as primeiras 5 entradas do DataFrame
df_atendimento_eletronico_final.head()

,UF,Município,PAE
0,,MIAMI,1
1,,NEW YORK,1
2,AC,ACRELANDIA,2
3,AC,ASSIS BRASIL,2
4,AC,BRASILEIA,3


In [901]:
# Como podem ter municípios presentes em uma tabela e não em outra, é usado o meio outer para manter informações das duas tabelas
# Junção das tabelas Agências e PA com base no município
df_banco = pd.merge(df_agencias_final, df_atendimento_final,  how="outer", on=["Município", "UF"])

In [902]:
# O mesmo procedimento é feito para a tabela resultante e PAE
df_banco = pd.merge(df_banco, df_atendimento_eletronico_final,  how="outer", on=["Município", "UF"])

In [903]:
# Como existem valores NaN, assume-se que são 0, uma vez que se não está presente em na tabela Agências,
# significa que não há nenhuma lá, o que interefere no nível de bancarização
df_banco = df_banco.fillna(0)

In [904]:
# Mostra as 5 primeiras entradas do DataFrame Banco, contendo as informações de agências e postos de atendimento sobre os municípios
df_banco.head()

,UF,Município,Agências,PA,PAE
0,AC,ACRELANDIA,1.0,3.0,2.0
1,AC,ASSIS BRASIL,1.0,1.0,2.0
2,AC,BRASILEIA,4.0,2.0,3.0
3,AC,BUJARI,1.0,0.0,2.0
4,AC,CRUZEIRO DO SUL,4.0,3.0,15.0


In [905]:
# Agora, com os quatro DataFrames df_municipios, df_idhm, df_celulares e df_banco, basta fazer joins para
# juntar todos eles, a fim de deixar todas as informações juntas
# Seguindo a mesma lógica das tabelas anteriores, são feitas joins do tipo outer
# Primeiro, ocorre o merge entre df_municipios, df_idhm e df_banco
# A condição é que o nome do município e o UF sejam iguais, já que existem municípios de mesmo nome no Brasil
data_frames = [df_municipios, df_idhm, df_banco]
df_bancarizacao = reduce(lambda  left,right: pd.merge(left,right,on=["Município", "UF"], how='outer'), data_frames)

In [906]:
# Em seguida, une-se df_bancarizacao e df_celulares baseado no UF 
df_bancarizacao = pd.merge(df_bancarizacao, df_celulares, how="inner", on="UF")

In [907]:
# Como agora só há Nan em informações que realmente são faltantes, essas entradas são retiradas do DataFrame
df_bancarizacao = df_bancarizacao.dropna()

# Reseta os índices após a remoçõ de algumas linhas
df_bancarizacao = df_bancarizacao.reset_index()

# Remove a coluna index, feita após o último método
df_bancarizacao = df_bancarizacao.drop(columns=["index"])

In [908]:
# Mostra as cinco primeiras entradas do DataFrame df_bancarizacao
df_bancarizacao.head()

,UF,Município,População,IDHM,Agências,PA,PAE,Teledensidade
0,SP,SAO PAULO,12325232.0,0.805,2170.0,1178.0,1864.0,1.5077
1,SP,GUARULHOS,1392121.0,0.763,101.0,113.0,149.0,1.5077
2,SP,CAMPINAS,1213792.0,0.805,203.0,153.0,139.0,1.5077
3,SP,SAO BERNARDO DO CAMPO,844483.0,0.805,90.0,95.0,115.0,1.5077
4,SP,SAO JOSE DOS CAMPOS,729737.0,0.807,75.0,101.0,90.0,1.5077


In [909]:
# Com as informações já tratadas e ponrtas para uso, agora é o momento de fazer os cálculos
# Primeiramente, declara-se a função contendo a fórmula do nível de bancarização
def bancarizacao(pop: np.float64, agencias: np.float64, pa: np.float64, pae: np.float64) -> float:
  nb = float((agencias + pa + pae) / pop)
  return nb

In [910]:
# Em seguida, define-se a função que retorna o coeficiente de ajusta populacional
def coef_pop(pop: np.float64) -> int:
  if 0 < pop <= 5_000:
    return 5
  elif 5_000 < pop <= 20_000:
    return 10
  elif 20_000 < pop <= 100_000:
    return 15
  elif 100_000 < pop <= 500_000:
    return 20
  elif pop > 500_000:
    return 25
  else:
    return None

In [911]:
# Por fim, define-se a função que calcula a probabilidade conversao
def prob_conversao(idhm: np.float64, cp: int, teledensidade: np.float64) -> int:
  pc = (idhm**cp)*(teledensidade/1.5)*100
  return pc

In [912]:
# Então, são feitos os cálculos
# Primeiramente, calcula-se o nível de bancarização

# Cria-se uma lista par guardar os valores
rows_bancarizacao = list()

# Loop para percorrer todas as linhas do DataFrame
for row in range(len(df_bancarizacao)):

  # Salva as informações de cada município nas variáveis
  pop = df_bancarizacao.at[row, "População"]
  agencias = df_bancarizacao.at[row, "Agências"]
  pa = df_bancarizacao.at[row, "PA"]
  pae = df_bancarizacao.at[row, "PAE"]

  # Calcula por meio da função bancarização
  nb = bancarizacao(pop, agencias, pa, pae)

  # Adiciona o valor ao vetor
  rows_bancarizacao.append(nb)

# Adiciona o vetor como uma nova coluna
df_bancarizacao["Bancarização"] = rows_bancarizacao

In [913]:
# Em seguida, calcula-se o coeficiente de ajuste populacional de cada município
df_bancarizacao["Ajuste Populacional"] = [coef_pop(pop_atual) for pop_atual in df_bancarizacao["População"]]

In [914]:
# Calcula-se a estimativa de clientes convertidos
# Cria-se uma lista para salvar os valores
estimativa_conversao = list()

# Um loop percorre as linhas do DataFrame
for row in range(len(df_bancarizacao)):
  idhm = df_bancarizacao.at[row, "IDHM"]
  cp = df_bancarizacao.at[row, "Ajuste Populacional"]
  teledensidade = df_bancarizacao.at[row, "Teledensidade"]
  pc = prob_conversao(idhm, cp, teledensidade)
  estimativa_conversao.append(pc)

df_bancarizacao["Estimativa Conversão"] = [f"{str(round(item, 2))}%" for item in estimativa_conversao]

In [915]:
# Por fim, calcula-se quantos clientes se convertem com base na estimativa, seguindo a mesma lógica
clientes_convertidos = list()

for row in range(len(df_bancarizacao)):
  pop = df_bancarizacao.at[row, "População"]
  estimativa = float(df_bancarizacao.at[row, "Estimativa Conversão"].replace("%", ""))
  cc = floor((pop*estimativa)/100)
  clientes_convertidos.append(cc)

df_bancarizacao["Clientes Convertidos"] = clientes_convertidos

In [916]:
# Organiza o DataFrame em ordem crescente de acordo com o nível de bancarização
df_bancarizacao = df_bancarizacao.sort_values("Bancarização")
df_bancarizacao = df_bancarizacao.reset_index()
df_bancarizacao = df_bancarizacao.drop(columns=["index"])

In [917]:
# Mostra os 5 municípios mais desbancarizados do Brasil, juntamente com todas as suas informações
df_bancarizacao.head()

,UF,Município,População,IDHM,Agências,PA,PAE,Teledensidade,Bancarização,Ajuste Populacional,Estimativa Conversão,Clientes Convertidos
0,AM,NOVO ARIPUANA,26046.0,0.554,1.0,0.0,0.0,0.9254,0.000038,15,0.01%,2
1,AM,NOVO AIRAO,19928.0,0.570,1.0,0.0,0.0,0.9254,0.000050,10,0.22%,43
2,AM,NOVA OLINDA DO NORTE,38026.0,0.558,1.0,0.0,1.0,0.9254,0.000053,15,0.01%,3
3,MA,TURIACU,35709.0,0.561,1.0,0.0,1.0,0.8365,0.000056,15,0.01%,3
4,BA,APORA,17788.0,0.548,0.0,0.0,1.0,0.9246,0.000056,10,0.15%,26


In [918]:
# Para listar as cidades necessárias para alcançar, cria-se um novo DataFrame com base em um contador, inicializado em 0
contador_clientes = 0

# Contador para controlar as linhas de df_bancarizacao
contador_rows = 0

# Lista que salvará os municípios de interesse
municipios_interesse = list()

# Lista com as colunas do DataFrame final
colunas = ["Município", "UF", "População", "Nível de Bancarização", "Cliente Convertidos"]

# Enquanto o contador é menor que 1000, o loop continua
# Como o contador será incrementado a cada município novo adicionado, teremos as cidades de interesse
while contador_clientes < 1000:
  municipio_atual = [
                     df_bancarizacao.at[contador_rows, "Município"],
                     df_bancarizacao.at[contador_rows, "UF"],
                     df_bancarizacao.at[contador_rows, "População"],
                     df_bancarizacao.at[contador_rows, "Bancarização"],
                     df_bancarizacao.at[contador_rows, "Clientes Convertidos"]
                     ]
  municipios_interesse.append(municipio_atual)

  # Soma a quantidade de clientes convertidos do município atual
  contador_clientes += df_bancarizacao.at[contador_rows, "Clientes Convertidos"]
  
  # Incrementa o contador
  contador_rows += 1

# Cria um novo DataFrame para salvar os municípios que se encaixam na proposta apresentada
df_municipios_interesse = pd.DataFrame(data=municipios_interesse, columns=colunas)

In [919]:
# Mostra as cidades de interesse, com um total de 1094 clientes convertidos
df_municipios_interesse

,Município,UF,População,Nível de Bancarização,Cliente Convertidos
0,NOVO ARIPUANA,AM,26046.0,0.000038,2
1,NOVO AIRAO,AM,19928.0,0.000050,43
2,NOVA OLINDA DO NORTE,AM,38026.0,0.000053,3
3,TURIACU,MA,35709.0,0.000056,3
4,APORA,BA,17788.0,0.000056,26
5,PILAO ARCADO,BA,35175.0,0.000057,0
6,CAUCAIA,CE,365212.0,0.000058,109
7,SALITRE,CE,16635.0,0.000060,23
8,MANAQUIRI,AM,33049.0,0.000061,9
9,SAO FELIX DO XINGU,PA,132138.0,0.000061,0


In [920]:
# Cria um arquivo .xlsx com o DataFrame final
df_municipios_interesse.to_excel("municipios_desbancarizacao.xlsx")